In [1]:
import os
import tensorflow as tf
import numpy as np
from IPython.display import Audio
from scipy.io import wavfile
from sklearn.decomposition import PCA

In [2]:
sample_rate, source_audio = wavfile.read("./data/donuts_hq.wav")
sample_start = sample_rate * 60
sample_end = sample_rate * 70
print("Sample rate:", sample_rate)

Sample rate: 32000


In [3]:
Audio(source_audio[sample_start:sample_end], rate=sample_rate)

In [4]:
def compress_with_pca(data, components, block_size=1024):
    samples = len(data)
    leftover = block_size - samples % block_size
    padded = np.lib.pad(data, (0, leftover), "constant", constant_values=0)
    
    reshaped = padded.reshape((len(padded) // block_size, block_size))
    
    pca = PCA(n_components=components)
    pca.fit(reshaped)
    
    transformed = pca.transform(reshaped)
    reconstructed = pca.inverse_transform(transformed).reshape((len(padded)))
    return pca, transformed, reconstructed

In [5]:
COMPONENT_COUNT = 200
PCA_BLOCK_SIZE = 3200 # 1/10 of the sample rate, which should be 32000 here

pca, transformed, constructed = compress_with_pca(
    source_audio, COMPONENT_COUNT, PCA_BLOCK_SIZE
)
print(f"PCA Transformed audio shape: {transformed.shape}")
Audio(constructed[sample_start:sample_end], rate=sample_rate)

PCA Transformed audio shape: (1741, 200)


In [6]:
pca_max = np.amax(transformed)
print(transformed.shape)
print(np.amax(transformed))
print(np.amin(transformed))
normalized = transformed / pca_max
print(normalized.shape)
print(np.amax(normalized))
print(np.amin(normalized))

(1741, 200)
972767.7704312496
-871977.7014758724
(1741, 200)
1.0
-0.8963883549403628


In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices(normalized)
train_dataset

<TensorSliceDataset shapes: (200,), types: tf.float64>

In [8]:
MAXIMUM_SEQUENCE_LENGTH = 8
sequences = train_dataset.batch(
    MAXIMUM_SEQUENCE_LENGTH + 1,
    drop_remainder=True
)
sequences

<BatchDataset shapes: (9, 200), types: tf.float64>

In [9]:
def split_input_target(sequence):
    input_data = sequence[:-1]
    target_data = sequence[1:]
    return input_data, target_data
    return sequence, sequence

prepared_dataset = sequences.map(split_input_target)
prepared_dataset

<MapDataset shapes: ((8, 200), (8, 200)), types: (tf.float64, tf.float64)>

In [10]:
BATCH_SIZE = 32
BUFFER_SIZE = 1024
shuffled_dataset = (
    prepared_dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
)

In [11]:
CHECKPOINT_DIR = "./training-checkpoints/music-generation-with-pca-and-deep-gru"

In [12]:
def build_model(rnn_units, batch_size):
    model =  tf.keras.Sequential([
        tf.keras.layers.InputLayer(
            batch_size=batch_size,
            input_shape=(None, COMPONENT_COUNT),
            dtype="float32",
        ),
        tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            stateful=True,
            recurrent_initializer="glorot_uniform",
            dtype="float32",
        ),
        tf.keras.layers.GRU(
            rnn_units // 2,
            return_sequences=True,
            stateful=True,
            recurrent_initializer="glorot_uniform",
            dtype="float32",
        ),
        tf.keras.layers.Dense(COMPONENT_COUNT, dtype="float32"),
    ])
    return model

In [13]:
latest_checkpoint = tf.train.latest_checkpoint(CHECKPOINT_DIR)

model = build_model(
    rnn_units=1024,
    batch_size=BATCH_SIZE,
)
model.build()
if latest_checkpoint:
    model.load_weights(latest_checkpoint)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (32, None, 1024)          3766272   
_________________________________________________________________
gru_1 (GRU)                  (32, None, 512)           2362368   
_________________________________________________________________
dense (Dense)                (32, None, 200)           102600    
Total params: 6,231,240
Trainable params: 6,231,240
Non-trainable params: 0
_________________________________________________________________


In [14]:
for input_batch, target_batch in shuffled_dataset.take(1):
    predictions = model(input_batch)
    print(predictions.shape, "# (batch_size, sequence_length, output)")
    print(predictions[0][0][:10])
    print(target_batch[0][0][:10])

(32, 8, 200) # (batch_size, sequence_length, output)
tf.Tensor(
[-0.00331659 -0.00518337 -0.00187783 -0.00676699 -0.00531852 -0.00139645
  0.00024159 -0.0036716   0.01133366  0.00855632], shape=(10,), dtype=float32)
tf.Tensor(
[-0.15528289 -0.09486162 -0.12488973  0.09779869  0.36410752 -0.06758649
 -0.35794271 -0.16075243 -0.12923337  0.02801628], shape=(10,), dtype=float64)


In [15]:
def loss(target, predicted):
    return tf.keras.losses.MSE(target, predicted)

In [16]:
batch_loss = loss(target_batch, predictions)
print("Predictions shape (batch_size, sequence_length, vocabulary_size)")
print(predictions.shape, "\n")
print("scalar_loss:", batch_loss.numpy().mean())

Predictions shape (batch_size, sequence_length, vocabulary_size)
(32, 8, 200) 

scalar_loss: 0.0016884693


In [17]:
model.compile(optimizer="adam", loss=loss)

In [29]:
checkpoint_prefix = os.path.abspath(
    os.path.join(CHECKPOINT_DIR, "ckpt_{epoch}")
)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    monitor="loss",
    save_best_only=True,
    period=100,
)

In [30]:
EPOCHS = 1000

In [31]:
history = model.fit(
    shuffled_dataset,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback],
)

Train for 6 steps
Epoch 1/1000
6/6 [==============================] - 0s 19ms/step - loss: 1.9953e-04
Epoch 2/1000
6/6 [==============================] - 0s 18ms/step - loss: 2.0033e-04
Epoch 3/1000
6/6 [==============================] - 0s 18ms/step - loss: 1.9974e-04
Epoch 4/1000
6/6 [==============================] - 0s 18ms/step - loss: 1.9725e-04
Epoch 5/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.9628e-04
Epoch 6/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.9878e-04
Epoch 7/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.9523e-04
Epoch 8/1000
6/6 [==============================] - 0s 16ms/step - loss: 1.9227e-04
Epoch 9/1000
6/6 [==============================] - 0s 17ms/step - loss: 2.0127e-04
Epoch 10/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.9664e-04
Epoch 11/1000
6/6 [==============================] - 0s 16ms/step - loss: 1.9512e-04
Epoch 12/1000
6/6 [==============================] - 0s 

6/6 [==============================] - 0s 16ms/step - loss: 1.5350e-04
Epoch 98/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.5399e-04
Epoch 99/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.5370e-04
Epoch 100/1000
6/6 [==============================] - 2s 283ms/step - loss: 1.5297e-04
Epoch 101/1000
6/6 [==============================] - 0s 16ms/step - loss: 1.5042e-04
Epoch 102/1000
6/6 [==============================] - 0s 16ms/step - loss: 1.5110e-04
Epoch 103/1000
6/6 [==============================] - 0s 18ms/step - loss: 1.4988e-04
Epoch 104/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.5266e-04
Epoch 105/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.5639e-04
Epoch 106/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.5732e-04
Epoch 107/1000
6/6 [==============================] - 0s 16ms/step - loss: 1.5736e-04
Epoch 108/1000
6/6 [==============================] - 0s 16ms/step - l

6/6 [==============================] - 0s 16ms/step - loss: 1.2169e-04
Epoch 193/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.2202e-04
Epoch 194/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.2121e-04
Epoch 195/1000
6/6 [==============================] - 0s 16ms/step - loss: 1.2046e-04
Epoch 196/1000
6/6 [==============================] - 0s 16ms/step - loss: 1.2020e-04
Epoch 197/1000
6/6 [==============================] - 0s 16ms/step - loss: 1.2080e-04
Epoch 198/1000
6/6 [==============================] - 0s 16ms/step - loss: 1.2255e-04
Epoch 199/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.2234e-04
Epoch 200/1000
6/6 [==============================] - 2s 278ms/step - loss: 1.2175e-04
Epoch 201/1000
6/6 [==============================] - 0s 19ms/step - loss: 1.2097e-04
Epoch 202/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.2175e-04
Epoch 203/1000
6/6 [==============================] - 0s 16ms/step -

6/6 [==============================] - 0s 16ms/step - loss: 9.9978e-05
Epoch 288/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.0309e-04
Epoch 289/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.0084e-04
Epoch 290/1000
6/6 [==============================] - 0s 16ms/step - loss: 1.0042e-04
Epoch 291/1000
6/6 [==============================] - 0s 16ms/step - loss: 1.0064e-04
Epoch 292/1000
6/6 [==============================] - 0s 16ms/step - loss: 9.9175e-05
Epoch 293/1000
6/6 [==============================] - 0s 17ms/step - loss: 1.0069e-04
Epoch 294/1000
6/6 [==============================] - 0s 16ms/step - loss: 1.0102e-04
Epoch 295/1000
6/6 [==============================] - 0s 16ms/step - loss: 1.0229e-04
Epoch 296/1000
6/6 [==============================] - 0s 16ms/step - loss: 1.0305e-04
Epoch 297/1000
6/6 [==============================] - 0s 16ms/step - loss: 1.0180e-04
Epoch 298/1000
6/6 [==============================] - 0s 16ms/step - 

6/6 [==============================] - 0s 16ms/step - loss: 8.2701e-05
Epoch 383/1000
6/6 [==============================] - 0s 16ms/step - loss: 8.3267e-05
Epoch 384/1000
6/6 [==============================] - 0s 16ms/step - loss: 8.3312e-05
Epoch 385/1000
6/6 [==============================] - 0s 16ms/step - loss: 8.3264e-05
Epoch 386/1000
6/6 [==============================] - 0s 16ms/step - loss: 8.3487e-05
Epoch 387/1000
6/6 [==============================] - 0s 16ms/step - loss: 8.4777e-05
Epoch 388/1000
6/6 [==============================] - 0s 16ms/step - loss: 8.2747e-05
Epoch 389/1000
6/6 [==============================] - 0s 16ms/step - loss: 8.2444e-05
Epoch 390/1000
6/6 [==============================] - 0s 15ms/step - loss: 8.2978e-05
Epoch 391/1000
6/6 [==============================] - 0s 16ms/step - loss: 8.2171e-05
Epoch 392/1000
6/6 [==============================] - 0s 16ms/step - loss: 8.2242e-05
Epoch 393/1000
6/6 [==============================] - 0s 16ms/step - 

...

6/6 [==============================] - 0s 16ms/step - loss: 6.2324e-05
Epoch 573/1000
6/6 [==============================] - 0s 17ms/step - loss: 6.2805e-05
Epoch 574/1000
6/6 [==============================] - 0s 16ms/step - loss: 6.4239e-05
Epoch 575/1000
6/6 [==============================] - 0s 17ms/step - loss: 6.2775e-05
Epoch 576/1000
6/6 [==============================] - 0s 17ms/step - loss: 6.2955e-05
Epoch 577/1000
6/6 [==============================] - 0s 17ms/step - loss: 6.3281e-05
Epoch 578/1000
6/6 [==============================] - 0s 17ms/step - loss: 6.3020e-05
Epoch 579/1000
6/6 [==============================] - 0s 16ms/step - loss: 6.2754e-05
Epoch 580/1000
6/6 [==============================] - 0s 17ms/step - loss: 6.1467e-05
Epoch 581/1000
6/6 [==============================] - 0s 16ms/step - loss: 6.0535e-05
Epoch 582/1000
6/6 [==============================] - 0s 17ms/step - loss: 6.0549e-05
Epoch 583/1000
6/6 [==============================] - 0s 18ms/step - 

6/6 [==============================] - 0s 17ms/step - loss: 5.4202e-05
Epoch 668/1000
6/6 [==============================] - 0s 17ms/step - loss: 5.3898e-05
Epoch 669/1000
6/6 [==============================] - 0s 16ms/step - loss: 5.4266e-05
Epoch 670/1000
6/6 [==============================] - 0s 16ms/step - loss: 5.3219e-05
Epoch 671/1000
6/6 [==============================] - 0s 16ms/step - loss: 5.3491e-05
Epoch 672/1000
6/6 [==============================] - 0s 16ms/step - loss: 5.5348e-05
Epoch 673/1000
6/6 [==============================] - 0s 16ms/step - loss: 5.6153e-05
Epoch 674/1000
6/6 [==============================] - 0s 17ms/step - loss: 5.4093e-05
Epoch 675/1000
6/6 [==============================] - 0s 17ms/step - loss: 5.4768e-05
Epoch 676/1000
6/6 [==============================] - 0s 17ms/step - loss: 5.4429e-05
Epoch 677/1000
6/6 [==============================] - 0s 16ms/step - loss: 5.6130e-05
Epoch 678/1000
6/6 [==============================] - 0s 17ms/step - 

6/6 [==============================] - 0s 17ms/step - loss: 4.8564e-05
Epoch 763/1000
6/6 [==============================] - 0s 16ms/step - loss: 4.9519e-05
Epoch 764/1000
6/6 [==============================] - 0s 16ms/step - loss: 4.8067e-05
Epoch 765/1000
6/6 [==============================] - 0s 16ms/step - loss: 4.8142e-05
Epoch 766/1000
6/6 [==============================] - 0s 16ms/step - loss: 4.7299e-05
Epoch 767/1000
6/6 [==============================] - 0s 16ms/step - loss: 4.7044e-05
Epoch 768/1000
6/6 [==============================] - 0s 16ms/step - loss: 4.6250e-05
Epoch 769/1000
6/6 [==============================] - 0s 16ms/step - loss: 4.6958e-05
Epoch 770/1000
6/6 [==============================] - 0s 17ms/step - loss: 4.6344e-05
Epoch 771/1000
6/6 [==============================] - 0s 17ms/step - loss: 4.6353e-05
Epoch 772/1000
6/6 [==============================] - 0s 16ms/step - loss: 4.6912e-05
Epoch 773/1000
6/6 [==============================] - 0s 17ms/step - 

6/6 [==============================] - 0s 16ms/step - loss: 4.3032e-05
Epoch 858/1000
6/6 [==============================] - 0s 16ms/step - loss: 4.3373e-05
Epoch 859/1000
6/6 [==============================] - 0s 17ms/step - loss: 4.3291e-05
Epoch 860/1000
6/6 [==============================] - 0s 16ms/step - loss: 4.2952e-05
Epoch 861/1000
6/6 [==============================] - 0s 16ms/step - loss: 4.3265e-05
Epoch 862/1000
6/6 [==============================] - 0s 16ms/step - loss: 4.3837e-05
Epoch 863/1000
6/6 [==============================] - 0s 16ms/step - loss: 4.4630e-05
Epoch 864/1000
6/6 [==============================] - 0s 17ms/step - loss: 4.4319e-05
Epoch 865/1000
6/6 [==============================] - 0s 16ms/step - loss: 4.4754e-05
Epoch 866/1000
6/6 [==============================] - 0s 17ms/step - loss: 4.3537e-05
Epoch 867/1000
6/6 [==============================] - 0s 17ms/step - loss: 4.3572e-05
Epoch 868/1000
6/6 [==============================] - 0s 16ms/step - 

6/6 [==============================] - 0s 16ms/step - loss: 4.0366e-05
Epoch 953/1000
6/6 [==============================] - 0s 17ms/step - loss: 4.0274e-05
Epoch 954/1000
6/6 [==============================] - 0s 17ms/step - loss: 4.1193e-05
Epoch 955/1000
6/6 [==============================] - 0s 16ms/step - loss: 4.0045e-05
Epoch 956/1000
6/6 [==============================] - 0s 17ms/step - loss: 3.9725e-05
Epoch 957/1000
6/6 [==============================] - 0s 16ms/step - loss: 3.9945e-05
Epoch 958/1000
6/6 [==============================] - 0s 17ms/step - loss: 3.8757e-05
Epoch 959/1000
6/6 [==============================] - 0s 17ms/step - loss: 3.8958e-05
Epoch 960/1000
6/6 [==============================] - 0s 17ms/step - loss: 3.8736e-05
Epoch 961/1000
6/6 [==============================] - 0s 16ms/step - loss: 3.9087e-05
Epoch 962/1000
6/6 [==============================] - 0s 16ms/step - loss: 3.8554e-05
Epoch 963/1000
6/6 [==============================] - 0s 17ms/step - 

In [32]:
model = build_model(
    rnn_units=1024,
    batch_size=1,
)
model.load_weights(tf.train.latest_checkpoint(CHECKPOINT_DIR))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_2 (GRU)                  (1, None, 1024)           3766272   
_________________________________________________________________
gru_3 (GRU)                  (1, None, 512)            2362368   
_________________________________________________________________
dense_1 (Dense)              (1, None, 200)            102600    
Total params: 6,231,240
Trainable params: 6,231,240
Non-trainable params: 0
_________________________________________________________________


In [33]:
def generate_output(model, input_eval, samples_to_generate=4000):
    generated_output = []
    
    model.reset_states()
    for i in range(samples_to_generate):
        predictions = model(input_eval)
        prediction = predictions[0, -1].numpy()
        input_eval = tf.expand_dims([prediction], 0)
        generated_output.append(prediction)
        if i % 50 == 0:
            print(f"Generated {i} / {samples_to_generate}")
    print("Done!")
    
    return generated_output

In [34]:
initial = list(shuffled_dataset.take(1))[0][0][0]
initial = tf.expand_dims(initial, 0)
Audio(pca.inverse_transform(initial * pca_max).flatten(), rate=sample_rate)

In [35]:
output = generate_output(model, initial, samples_to_generate=200)
output = np.array(output) * pca_max
# model.reset_states()
# output = model(initial) * pca_max

Generated 0 / 200
Generated 50 / 200
Generated 100 / 200
Generated 150 / 200
Done!


In [36]:
output_reconstructed = pca.inverse_transform(output).flatten()
output_reconstructed *= pca_max
Audio(output_reconstructed, rate=sample_rate)